In [10]:
!pip install yolov5 --user

  Obtaining dependency information for google-cloud-secret-manager from https://files.pythonhosted.org/packages/e2/64/d654d419dac6e313de993c58b821f6dbbf757cf61dfd7c37de4443dac550/google_cloud_secret_manager-2.16.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python>=4.1.1 from https://files.pythonhosted.org/packages/b7/8a/b2f7e1a434d56bf1d7570fc5941ace0847404e1032d7f1f0b8fed896568d/opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Obtaining dependency information for opencv-python-headless==4.8.0.74 from https://files.pythonhosted.org/packages/76/02/f128517f3ade4bb5f71e2afd8461dba70e3f466ce745fa1fd1fade9ad1b7/opencv_python_headless-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.w

In [21]:
!pip uninstall -y opencv-python opencv-python-headless && pip install opencv-python-headless Pillow==9.5.0 --user

Found existing installation: opencv-python-headless 4.8.1.78
Uninstalling opencv-python-headless-4.8.1.78:
  Successfully uninstalled opencv-python-headless-4.8.1.78
  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/9d/d7/e2aaf344254292d2046f9984b54212e4e7d69a57d30ae15e7294840710f6/opencv_python_headless-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached opencv_python_headless-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 46.4 MB/s eta 0:00:0000:0100:01
Using cached opencv_python_headless-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (49.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sahi 0.11.14 requires opencv-python>=4.2.0.32, which is not insta

In [19]:
!pip install google-cloud-secret-manager --user

In [16]:
from google.cloud import secretmanager
secrets = secretmanager.SecretManagerServiceClient()
from roboflow import Roboflow
import os
import yolov5
from yolov5 import train, val, detect, export
import yaml

In [13]:
cmdOut = !curl -s -H 'Metadata-Flavor:Google' 'http://metadata.google.internal/computeMetadata/v1/project/project-id'
project_id = cmdOut[0]
roboflow_secret = f"projects/{project_id}/secrets/roboflow-api-key/versions/latest"
roboflow_api_key = secrets.access_secret_version(secretmanager.AccessSecretVersionRequest(name=roboflow_secret)).payload.data.decode("ascii")

In [14]:
rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace("deneme-iq93l").project("chessv1-sswl9")
dataset = project.version(4).download("yolov5", location="datasets/chess")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to datasets/chess in yolov5pytorch:: 100%|██████████| 39002/39002 [00:05<00:00, 6574.03it/s] 


In [18]:
with open("datasets/chess/data.yaml", "r") as data_yaml_file:
    data_yaml = yaml.safe_load(data_yaml_file)
data_yaml['test'] = os.path.join(os.getcwd(), "datasets/chess/test/images")
data_yaml['train'] = os.path.join(os.getcwd(), "datasets/chess/train/images")
data_yaml['val'] = os.path.join(os.getcwd(), "datasets/chess/valid/images")
with open("datasets/chess/data.yaml", "w") as data_yaml_file:
    yaml.dump(data_yaml, data_yaml_file)


In [ ]:
!~/.local/bin/yolov5 train --weights=yolov5x.pt --data=datasets/chess/data.yaml --batch-size=16

train: weights=yolov5x.pt, cfg=, data=datasets/chess/data.yaml, hyp=.local/lib/python3.10/site-packages/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, bbox_interval=-1, artifact_alias=latest, neptune_token=None, neptune_project=None, neptune_resume_id=None, s3_upload_dir=None, upload_dataset=False, hf_model_id=None, hf_token=None, hf_private=False, hf_dataset_id=None, roboflow_token=None, roboflow_upload=False
requirements: /home/jupyter/.local/lib/python3.10/site-packages/requirements.txt not found, check failed.
YOLOv5 🚀 2023-10-20 Python-3.10.12 torch-2.0.0+cu